In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
from data import ModelNet40,download,load_data

# from data import ModelNet40Convex,download,load_data
from data_convex import ModelNet40Convex,download,load_data
import numpy as np
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from geomloss import SamplesLoss
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import numpy as np
from matplotlib import pyplot as plt
import pickle
import warnings
import math
import time
from datetime import datetime
from collections import OrderedDict
from scipy.optimize import linear_sum_assignment
from torch import jit
%matplotlib inline
import os
from torch.utils.data import Dataset, TensorDataset

/workspace/ConvexICP/data_convex.py:46: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File(h5_name)


In [3]:
# !pip install geomloss

In [4]:
train_losses = []
test_results_exp = []
test_accs = []

In [5]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 2 every 30 epochs"""
    lrt = lr * (0.5 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lrt

In [6]:
device = torch.device("cuda:2")

In [7]:
data = ModelNet40Convex(num_points=2048, partition='train', gaussian_noise=False,
                       unseen=False, factor=4)
data_test = ModelNet40(num_points=2048, partition='test', gaussian_noise=False,
                       unseen=False, factor=4)
train_loader = DataLoader(data, batch_size=32, shuffle=True, drop_last=True)
test_loader = DataLoader(data_test, batch_size=1, shuffle=True, drop_last=True)

/workspace/ConvexICP/data_convex.py:46: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File(h5_name)
/workspace/ConvexICP/data.py:36: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File(h5_name)


In [8]:
# # # Original
# data = ModelNet40(num_points=2048, partition='train', gaussian_noise=False,
#                        unseen=False, factor=4)
# data_test = ModelNet40(num_points=2048, partition='test', gaussian_noise=False,
#                        unseen=False, factor=4)
# train_loader = DataLoader(data, batch_size=128, shuffle=True, drop_last=True)
# test_loader = DataLoader(data_test, batch_size=1, shuffle=True, drop_last=True)

In [9]:
print('Test Loader')
sample=data[0]

len(sample)
for i,e in enumerate(sample):
    try:
        print(i,e.shape)
    except:
        print(i,len(e))

Test Loader
0 (3, 2048)
1 (3, 2048)
2 (3, 2048)
3 (3, 2048)
4 2
5 2
6 2
7 2


In [10]:
def train_eval_icp(model,device, train_loader,test_loader, optimizer,criterion, epoch, eval_mode='on'):
    global best
    model.train()
    final_cal = nn.Sigmoid()
    train_error_logs = []
    t1 = datetime.now()
    test_loss_log = []
    test_acc_log = []
    train_corrects = 0
    
    for batch_idx, (src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba) in enumerate(train_loader):
        src = src.to(device)
        target = target.to(device)
        rotation_ab = rotation_ab.to(device)
        translation_ab = translation_ab.to(device)
        rotation_ba = rotation_ba.to(device)
        translation_ba = translation_ba.to(device)
        optimizer.zero_grad()
        output_train = model(src,target,rotation_ab,translation_ab)
        loss = criterion(output_train, target)
        loss.backward()
        optimizer.step()
        #pred_train = output_train.argmax(dim=1, keepdim=True) # get the index of the max log-probability
        #train_corrects += pred_train.eq(target.view_as(pred_train)).sum().item()
        train_error_logs.append(loss.item())
        
        if (batch_idx+0) % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTime: {:.2f}'.format(
                epoch, (batch_idx+0)* len(target), len(train_loader.dataset),
                100. * (batch_idx+0) / len(train_loader), loss,(datetime.now()-t1).total_seconds()))

        
    print('Train Epoch: {} Accuracy: {}/{} ({:.2f}%)\n'.format(
                epoch, train_corrects, len(train_loader.dataset),
                100. * train_corrects / len(train_loader.dataset)))
    
    model.eval()
    test_loss = 0
    
    with torch.no_grad():
        for src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba in enumerate(test_loader):
            src = src.to(device)
            target = target.to(device)
            rotation_ab = rotation_ab.to(device)
            translation_ab = translation_ab.to(device)
            rotation_ba = rotation_ba.to(device)
            translation_ba = translation_ba.to(device)
            output_test = model(src,target,rotation_ab,translation_ab)
            test_loss += criterion(output_test, target_test).item()  # sum up batch loss
    test_loss /= len(test_loader.dataset)
    test_loss_log.append(test_loss)
    print('Test set: Average loss: {:.8f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
            test_loss, correct_test, len(test_loader.dataset),
            100. * correct_test / len(test_loader.dataset)))
    return train_error_logs,test_loss_log,test_acc_log

In [11]:
class PointNet(nn.Module):
    def __init__(self, emb_dims=512):
        super(PointNet, self).__init__()
        self.conv1 = nn.Conv1d(3, 64, kernel_size=1, bias=False)
        self.conv2 = nn.Conv1d(64, 64, kernel_size=1, bias=False)
        self.conv3 = nn.Conv1d(64, 64, kernel_size=1, bias=False)
        self.conv4 = nn.Conv1d(64, 128, kernel_size=1, bias=False)
        self.conv5 = nn.Conv1d(128, emb_dims, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(64)
        self.bn3 = nn.BatchNorm1d(64)
        self.bn4 = nn.BatchNorm1d(128)
        self.bn5 = nn.BatchNorm1d(emb_dims)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.relu(self.bn5(self.conv5(x)))
        return x

In [12]:
class CPNet(nn.Module):
    def __init__(self, emb_dims=512):
        super(CPNet, self).__init__()
        self.base_net = PointNet()
        self.loss = SamplesLoss(loss="sinkhorn", p=2, blur=.05)

        
    def forward(self, x1, x2, R, T):
        
        trans = torch.matmul(R, x1) + T.unsqueeze(2)
        emb1 = self.base_net(trans)
        emb2 = self.base_net(x2)
        
        dist = self.loss(emb1,emb2)

        return dist

In [13]:
torch.Tensor(1,3,3)

tensor([[[-7.5352e+22,  4.5769e-41,  9.8596e+02],
         [ 3.0653e-41,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00]]])

In [14]:
# 

In [15]:
class CPEvalNet(nn.Module):
    def __init__(self, base_net, emb_dims=512):
        super(CPEvalNet, self).__init__()
        self.base_net = PointNet()
        self.base_net.load_state_dict(base_net.state_dict())
        for param in self.base_net:
            param.require_grad = False
        self.rotation = nn.Parameter(torch.Tensor(1,3,3))
        self.translation = nn.Parameter(torch.Tensor(1,3))
        nn.init.kaiming_uniform_(self.rotation, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.translation, a=math.sqrt(5))
        self.loss = SamplesLoss(loss="sinkhorn", p=2, blur=.05)

        
    def forward(self, x1, x2):
        
        trans = torch.matmul(self.rotation, x1) + self.translation.unsqueeze(2)
        emb1 = self.base_net(trans)
        emb2 = self.base_net(x2)
        
        dist = self.loss(emb1,emb2)

        return dist

In [16]:
def dummy_loss(output, target):
    loss = torch.mean(output)
    return loss

In [17]:
lr = 1e-3
momentum = 0.9
weight_decay = 5e-4

In [18]:
model = CPNet().to(device)

In [19]:
for batch_idx, content in enumerate(train_loader):
    (src, pointcloud1, pointcloud2, pointcloud3,mat1,mat2,mat3_0,mat3_1)= content
    print(batch_idx)
    break

0


In [20]:
# model(src,pointcloud1,R_ab1,translation_ab1)

# type(pointcloud1),type(R_ab1),type(translation_ab1),type(src),model

In [21]:
criterion = dummy_loss
optimizer = torch.optim.SGD(model.parameters(), lr,
                                momentum=momentum,
                                weight_decay=weight_decay)
best=0

In [22]:
model

CPNet(
  (base_net): PointNet(
    (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,), bias=False)
    (conv2): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
    (conv3): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
    (conv4): Conv1d(64, 128, kernel_size=(1,), stride=(1,), bias=False)
    (conv5): Conv1d(128, 512, kernel_size=(1,), stride=(1,), bias=False)
    (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (loss): SamplesLoss()
)

In [23]:
# print(src.shape,pointcloud1.shape,R_ab1.shape,translation_ab1.shape)

# trans = torch.matmul(R, x1) + T.unsqueeze(2)
# trans = torch.matmul(R_ab1, src) + translation_ab1.unsqueeze(2)

# translation_ab1.unsqueeze(2).shape
# print(src.shape,target.shape,rotation_ab.shape, translation_ab.shape)
# src
# rotation_ab
# .float()
# trans = torch.matmul(R_ab1.float(), src.float()) + translation_ab1.float().unsqueeze(2)
# trans.shape

In [24]:
# global best
# model.train()
# final_cal = nn.Sigmoid()
# train_error_logs = []
# t1 = datetime.now()
# test_loss_log = []
# test_acc_log = []

# for batch_idx, content in enumerate(train_loader):
#     (src, pointcloud1, pointcloud2, pointcloud3,mat1,mat2,mat3_0,mat3_1)=content
#     src=src.to(device)
# #         target = target.to(device)
#     pointcloud1=pointcloud1.to(device)
#     pointcloud2=pointcloud2.to(device)
#     pointcloud3=pointcloud3.to(device)


#     optimizer.zero_grad()

#     translation_ab1,R_ab1=mat1[0],mat1[1]

#     translation_ab2,R_ab2=mat2[0],mat2[1]

#     translation_ab3_0,R_ab3_0=mat3_0[0],mat3_0[1]
#     translation_ab3_1,R_ab3_1=mat3_1[0],mat3_1[1]
#     translation_ab3=translation_ab3_0+translation_ab3_1
#     R_ab3=R_ab3_0.matmul(R_ab3_1)

#     translation_ab1 = translation_ab1.to(device)
#     translation_ab2 = translation_ab2.to(device)
#     translation_ab3 = translation_ab3.to(device)
#     R_ab1 = R_ab1.to(device)
#     R_ab2 = R_ab2.to(device)
#     R_ab3 = R_ab3.to(device)

    
    
    
#     output_train1 = model(src.float(),pointcloud1.float(),R_ab1.float(),translation_ab1.float())    
#     output_train2 = model(src.float(),pointcloud2.float(),R_ab2.float(),translation_ab2.float())
#     output_train3 = model(src.float(),pointcloud3.float(),R_ab3.float(),translation_ab3.float())


#     loss1 = criterion(output_train1, pointcloud1)
#     loss2 = criterion(output_train2, pointcloud2)
#     loss3 = criterion(output_train3, pointcloud3)

#     loss1.backward()
#     loss2.backward()
#     loss3.backward()
    
    
#     optimizer.step()
#     #pred_train = output_train.argmax(dim=1, keepdim=True) # get the index of the max log-probability
#     #train_corrects += pred_train.eq(target.view_as(pred_train)).sum().item()
# #         train_error_logs.append(loss.item())
#     train_error_logs.append(loss1.item())
#     train_error_logs.append(loss2.item())
#     train_error_logs.append(loss3.item())

#     if (batch_idx+0) % 20 == 0:
#         print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTime: {:.2f}'.format(
#             epoch, (batch_idx+0)* len(target), len(train_loader.dataset),
#             100. * (batch_idx+0) / len(train_loader), loss,(datetime.now()-t1).total_seconds()))


# print('Train Epoch: {} Accuracy: {}/{} ({:.2f}%)\n'.format(
#             epoch, train_corrects, len(train_loader.dataset),
#             100. * train_corrects / len(train_loader.dataset)))

# model.eval()
# test_loss = 0    

In [25]:


def train_eval_convexity(model,device, train_loader,test_loader, optimizer,criterion, epoch, eval_mode='on'):
    global best
    model.train()
    final_cal = nn.Sigmoid()
    train_error_logs = []
    t1 = datetime.now()
    test_loss_log = []
    test_acc_log = []
    train_corrects = 0
    
#     for batch_idx, (src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba) in enumerate(train_loader):
    for batch_idx, content in enumerate(train_loader):
        (src, pointcloud1, pointcloud2, pointcloud3,mat1,mat2,mat3_0,mat3_1)=content
        src=src.to(device).float()
#         target = target.to(device)
        pointcloud1=pointcloud1.to(device).float()
        pointcloud2=pointcloud2.to(device).float()
        pointcloud3=pointcloud3.to(device).float()
        

        optimizer.zero_grad()
        
        translation_ab1,R_ab1=mat1[0],mat1[1]

        translation_ab2,R_ab2=mat2[0],mat2[1]

        translation_ab3_0,R_ab3_0=mat3_0[0],mat3_0[1]
        translation_ab3_1,R_ab3_1=mat3_1[0],mat3_1[1]
        translation_ab3=translation_ab3_0+translation_ab3_1
        R_ab3=R_ab3_0.matmul(R_ab3_1)
        
        translation_ab1 = translation_ab1.to(device).float()
        translation_ab2 = translation_ab2.to(device).float()
        translation_ab3 = translation_ab3.to(device).float()
        R_ab1 = R_ab1.to(device).float()
        R_ab2 = R_ab2.to(device).float()
        R_ab3 = R_ab3.to(device).float()
        
        
        output_train1 = model(src,pointcloud1,R_ab1,translation_ab1)
        output_train2 = model(src,pointcloud2,R_ab2,translation_ab2)
        output_train3 = model(src,pointcloud3,R_ab3,translation_ab3)
        
        
        loss1 = criterion(output_train1, pointcloud1)
        loss2 = criterion(output_train2, pointcloud2)
        loss3 = criterion(output_train3, pointcloud3)
        
        loss1.backward()
        loss2.backward()
        loss3.backward()
        
        
        loss=(loss1+loss2+loss3)
        optimizer.step()
        #pred_train = output_train.argmax(dim=1, keepdim=True) # get the index of the max log-probability
        #train_corrects += pred_train.eq(target.view_as(pred_train)).sum().item()
#         train_error_logs.append(loss.item())
        train_error_logs.append(loss1.item())
        train_error_logs.append(loss2.item())
        train_error_logs.append(loss3.item())
        
        if (batch_idx+0) % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTime: {:.2f}'.format(
                epoch, (batch_idx+0)* len(src), len(train_loader.dataset),
                100. * (batch_idx+0) / len(train_loader), loss,(datetime.now()-t1).total_seconds()))

        
    print('Train Epoch: {} Accuracy: {}/{} ({:.2f}%)\n'.format(
                epoch, train_corrects, len(train_loader.dataset),
                100. * train_corrects / len(train_loader.dataset)))
    
    model.eval()
    test_loss = 0
    
    with torch.no_grad():
        for src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba in enumerate(test_loader):
            src = src.to(device)
            target = target.to(device)
            rotation_ab = rotation_ab.to(device)
            translation_ab = translation_ab.to(device)
            rotation_ba = rotation_ba.to(device)
            translation_ba = translation_ba.to(device)
            output_test = model(src,target,rotation_ab,translation_ab)
            test_loss += criterion(output_test, target_test).item()  # sum up batch loss
    test_loss /= len(test_loader.dataset)
    test_loss_log.append(test_loss)
    print('Test set: Average loss: {:.8f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
            test_loss, correct_test, len(test_loader.dataset),
            100. * correct_test / len(test_loader.dataset)))
    return train_error_logs,test_loss_log,test_acc_log

In [26]:
# Convex Training procedure
train_time= []
train_loss = []
train_losses.append(train_loss)
tests = []
test_acc = []
test_results_exp.append(tests)
test_accs.append(test_acc)
ratio = 0
for epoch in range(1,30):
    adjust_learning_rate(optimizer, epoch)
    t1 = datetime.now()
    train_error,test_error,test_acc_this = train_eval_convexity(model,device, train_loader, test_loader, optimizer,criterion,epoch,'on')
    train_loss.extend(train_error)
    tests.extend(test_error)
    test_acc.extend(test_acc_this)
    train_time.append((datetime.now()-t1).total_seconds())
    print((datetime.now()-t1).total_seconds())

Train Epoch: 1 [0/9840 (0%)]	Loss: 1886.084839	Time: 1.25
Train Epoch: 1 [640/9840 (7%)]	Loss: 2.709973	Time: 9.96
Train Epoch: 1 [1280/9840 (13%)]	Loss: 0.017026	Time: 18.37
Train Epoch: 1 [1920/9840 (20%)]	Loss: 0.001057	Time: 26.44
Train Epoch: 1 [2560/9840 (26%)]	Loss: 0.011959	Time: 34.52
Train Epoch: 1 [3200/9840 (33%)]	Loss: 0.001022	Time: 42.56
Train Epoch: 1 [3840/9840 (39%)]	Loss: 0.000288	Time: 50.45
Train Epoch: 1 [4480/9840 (46%)]	Loss: 0.000710	Time: 58.44
Train Epoch: 1 [5120/9840 (52%)]	Loss: 0.000353	Time: 66.47
Train Epoch: 1 [5760/9840 (59%)]	Loss: 0.001293	Time: 74.53
Train Epoch: 1 [6400/9840 (65%)]	Loss: 0.000234	Time: 82.53
Train Epoch: 1 [7040/9840 (72%)]	Loss: 0.000118	Time: 90.68
Train Epoch: 1 [7680/9840 (78%)]	Loss: 0.000062	Time: 98.68
Train Epoch: 1 [8320/9840 (85%)]	Loss: 0.000184	Time: 106.58
Train Epoch: 1 [8960/9840 (91%)]	Loss: 0.000103	Time: 114.74
Train Epoch: 1 [9600/9840 (98%)]	Loss: 0.000042	Time: 122.66
Train Epoch: 1 Accuracy: 0/9840 (0.00%)



ValueError: not enough values to unpack (expected 8, got 2)

In [26]:
# Original Training
criterion = dummy_loss
optimizer = torch.optim.SGD(model.parameters(), lr,
                                momentum=momentum,
                                weight_decay=weight_decay)
best=0

In [27]:
train_time= []
train_loss = []
train_losses.append(train_loss)
tests = []
test_acc = []
test_results_exp.append(tests)
test_accs.append(test_acc)
ratio = 0
for epoch in range(1,20):
    adjust_learning_rate(optimizer, epoch)
    t1 = datetime.now()
    train_error,test_error,test_acc_this = train_eval_icp(model,device, train_loader, test_loader, optimizer,criterion,epoch,'on')
    train_loss.extend(train_error)
    tests.extend(test_error)
    test_acc.extend(test_acc_this)
    train_time.append((datetime.now()-t1).total_seconds())
    print((datetime.now()-t1).total_seconds())

NameError: name 'train_losses' is not defined

In [35]:
for batch_idx, (src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba) in enumerate(train_loader):
    break

(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)
(3, 2048) (2048, 3)


ValueError: not enough values to unpack (expected 8, got 4)

In [34]:
translation_ab.shape

torch.Size([128, 3])